# Read this first 

* each comment and text line added here is important and relevant , dont skim through just the code
* if the comment asks you to revisit some concepts/modules , do so
* this is not complete code on its own , you need to fill in your own code to eventually start building a model

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [2]:
datafile_train="/Users/lalitsachan/Dropbox/March onwards/PDS V2/Projects/P5/Property_train.csv"
datafile_test="/Users/lalitsachan/Dropbox/March onwards/PDS V2/Projects/P5/Property_test_share.csv"
bd_train=pd.read_csv(datafile_train)
bd_test=pd.read_csv(datafile_test)

In [3]:
bd_train.head()

,Junk,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,...,Region,PriceIndex5,SubModel,Facade,State,NormalisedPopulation,BuildYear,RegionType,PropertyAge,PriceIndex2
0,0,6LLJ,14674.0,9/7/2010,UT7W,8270,CAT3,B,missing,13143.0,...,A,9022.0,6E9G,GREEN,MD,42077,2008,A,2,10692.0
1,1,XZ9F,4172.0,1/7/2009,PRN0,3890,CAT2,B,missing,3461.0,...,A,2742.0,FIVQ,SILVER,OK,77258,2002,A,7,3615.0
2,0,QWEV,8210.0,6/23/2010,A9RA,5900,CAT1,B,missing,6922.0,...,A,3768.0,4K0H,SILVER,SC,94514,2004,A,6,5805.0
3,0,MK2K,7309.0,3/19/2009,RRJQ,7460,OTHER,A,missing,6300.0,...,A,5370.0,ZLXY,WHITE,NC,82302,2002,C,7,6444.0
4,0,GK4G,9182.0,3/3/2010,HE5A,6400,OTHER,B,missing,8330.0,...,A,4803.0,FIVQ,SILVER,SC,49176,2007,B,3,6286.0


In [4]:
# we are going to drop date column 'ListDate', you should extract components 
# from it and use them as features . create cyclic features from cyclic components 



In [5]:
target='Junk'

In [6]:
## look at the columns carefully and see what variables are stored as categorical but 
## need to be treated as numeric [PriceIndex8] . And some which are stored as numeric but are representing 
## categorical information [such as Zip]

bd_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62035 entries, 0 to 62034
Data columns (total 31 columns):
Junk                     62035 non-null int64
InteriorsStyle           62035 non-null object
PriceIndex8              62035 non-null object
ListDate                 62035 non-null object
Material                 62035 non-null object
PriceIndex9              62035 non-null int64
Agency                   62035 non-null object
AreaIncomeType           62035 non-null object
EnvRating                62035 non-null object
PriceIndex7              62035 non-null object
ExpeditedListing         62035 non-null int64
PriceIndex4              62035 non-null object
PriceIndex1              62035 non-null object
PriceIndex6              62035 non-null object
PRIMEUNIT                62035 non-null object
Channel                  62035 non-null object
Zip                      62035 non-null int64
InsurancePremiumIndex    62035 non-null int64
PlotType                 62035 non-null object
Arc

In [7]:
bd_train.head()

,Junk,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,...,Region,PriceIndex5,SubModel,Facade,State,NormalisedPopulation,BuildYear,RegionType,PropertyAge,PriceIndex2
0,0,6LLJ,14674.0,9/7/2010,UT7W,8270,CAT3,B,missing,13143.0,...,A,9022.0,6E9G,GREEN,MD,42077,2008,A,2,10692.0
1,1,XZ9F,4172.0,1/7/2009,PRN0,3890,CAT2,B,missing,3461.0,...,A,2742.0,FIVQ,SILVER,OK,77258,2002,A,7,3615.0
2,0,QWEV,8210.0,6/23/2010,A9RA,5900,CAT1,B,missing,6922.0,...,A,3768.0,4K0H,SILVER,SC,94514,2004,A,6,5805.0
3,0,MK2K,7309.0,3/19/2009,RRJQ,7460,OTHER,A,missing,6300.0,...,A,5370.0,ZLXY,WHITE,NC,82302,2002,C,7,6444.0
4,0,GK4G,9182.0,3/3/2010,HE5A,6400,OTHER,B,missing,8330.0,...,A,4803.0,FIVQ,SILVER,SC,49176,2007,B,3,6286.0


In [8]:
cat_to_numeric=['PriceIndex'+str(i) for i in range(1,10)]

In [9]:
bd_train['data']='train'
bd_test['data']='test'

all_data=pd.concat([bd_train,bd_test],0,sort=False)

In [10]:
for col in cat_to_numeric:
    all_data[col]=pd.to_numeric(all_data[col],errors='coerce')
    all_data[col]=all_data[col].fillna(all_data.loc[all_data['data']=='train',col].median())

In [11]:
cat_cols=['InteriorsStyle', 'ListDate', 'Material', 'Agency', 'AreaIncomeType',
       'EnvRating', 'PRIMEUNIT', 'Channel', 'PlotType', 'Architecture',
       'Region', 'SubModel', 'Facade', 'State', 'RegionType','Zip']

In [12]:
for col in cat_cols:
    k=all_data[col].value_counts()
    cats=k[k>=1000].index[:-1]
    
    for cat in cats:
        name=col+'_'+str(cat)
        all_data[name]=(all_data[col]==cat).astype(int)
        
    del all_data[col]
# you can try more lenient frequency cutoff , which will ofcourse 

In [13]:
all_data.shape

(72983, 125)

,Junk,PriceIndex8,PriceIndex9,PriceIndex7,ExpeditedListing,PriceIndex4,PriceIndex1,PriceIndex6,InsurancePremiumIndex,PriceIndex3,...,Zip_76040,Zip_77041,Zip_78754,Zip_80011,Zip_73108,Zip_28625,Zip_91752,Zip_78219,Zip_30272,Zip_32124
0,0.0,14674.0,8270,13143.0,0,14224.0,9217.0,10387.0,623,13108.0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,4172.0,3890,3461.0,0,4404.0,2958.0,3400.0,1689,3695.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,8210.0,5900,6922.0,0,8952.0,4568.0,4862.0,2351,7730.0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,7309.0,7460,6300.0,0,7460.0,5361.0,6305.0,1933,6290.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,9182.0,6400,8330.0,0,9846.0,5428.0,5718.0,482,8583.0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
x_train=all_data.drop([target,'data'],1)[all_data['data']=='train']
y_train=all_data[target][all_data['data']=='train']
x_test=all_data.drop([target,'data'],1)[all_data['data']=='test']

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

In [20]:
params={'penalty':['l1','l2'],'class_weight':['balanced',None],
        'C':np.linspace(0.01,100,100)}


In [21]:
model=LogisticRegression()

# tune more complex algos for improving performance

In [22]:
rs=RandomizedSearchCV(model,param_distributions=params,n_iter=10,
                      scoring='roc_auc',cv=10,n_jobs=-1,verbose=20)

In [23]:
rs.fit(x_train,y_train)

# this takes quite awhile to run on my machine , more complex algos 
# are going to take much more time , couple hrs may be for parameter tuning 
# do preliminary feature selection first a simple random forest and then do parameter tuning

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   22.2s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'penalty': ['l1', 'l2'], 'class_weight': ['balanced', None], 'C': array([1.000e-02, 1.020e+00, ..., 9.899e+01, 1.000e+02])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=20)

In [25]:
submissions=pd.DataFrame({'Junk':rs.predict_proba(x_test)[:,1]})

In [26]:
submissions.to_csv('sample_submission.csv',index=False)